<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_4_tuning_bigtweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    982      0 --:--:-- --:--:-- --:--:--   982
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-4.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-4.validation.csv",header=None)
valid.head()

,0,1
0,6,diastolic dysfunction
1,8,ARTHRITIS
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-4.test.csv",header=None)
test.head()

,0,1
0,4,purple sploches around waist
1,6,stage 1 diastolic dysfunction
2,8,arthritis
3,12,shingles
4,16,flu


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_bigtweet_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [18]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [19]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'rt',
 'to',
 'the',
 'for',
 'a',
 'in',
 'you',
 'is',
 'of',
 'on',
 'and',
 'disease',
 'amp',
 'i',
 'gluten',
 'with',
 'free',
 'your',
 'it',
 'my',
 'have',
 'new',
 'this',
 'are',
 'about',
 'glutenfree',
 'our',
 'what',
 'do',
 'nt',
 'celiac',
 'out',
 'be',
 'at',
 'from',
 'now',
 'not',
 'or',
 'that',
 'we',
 'how',
 'food',
 'via',
 'can',
 'right',
 'tip',
 'its',
 'all',
 'more',
 'if',
 'w',
 'use',
 'dblclick',
 'products',
 'so',
 'pic',
 'diet',
 'as',
 'why',
 'up',
 'know',
 'check',
 'here',
 'review',
 'me',
 'by',
 'love',
 'get',
 'just',
 'gfb',
 'thanks',
 'm',
 'was',
 'an',
 're',
 'great',
 'could',
 'recipe',
 'who',
 's',
 'has',
 'posted',
 'but',
 'will',
 'post',
 'help',
 'there',
 'may',
 'us',
 '\r \n ',
 'heres',
 'read',
 'make',
 'archives',
 '30',
 'some',
 'test',
 'people',
 'eat',
 'intolerance',
 'when',
 'going',
 'life',
 'need',
 'does',
 'je

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.136908,4.210674,0.272577,04:23


In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.051464,3.677505,0.333897,04:21


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.769664,3.634775,0.339892,04:29


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.710653,3.595909,0.344898,04:24


In [0]:
learn.save_encoder('bigtweet_fold4_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold4_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.643535,4.327496,0.298309,01:19


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.869188,3.828566,0.371981,01:29


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.524093,3.667972,0.396135,03:40


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.398040,3.438875,0.413043,03:54
1,3.172596,3.346619,0.427536,04:00


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.030760,3.286966,0.427536,03:58
1,3.034153,3.279406,0.435990,03:42


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.984600,3.181027,0.437198,03:46
1,2.782577,3.126896,0.446860,03:58


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.772133,3.133345,0.450483,03:54
1,2.691344,3.075701,0.446860,03:47


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.645781,3.057143,0.467391,03:58
1,2.596673,3.050850,0.461353,03:45


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.525389,3.018938,0.456522,03:40
1,2.535276,3.054662,0.458937,03:50
2,2.512902,3.042873,0.464976,04:04
3,2.390213,2.969960,0.467391,03:46


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.466293,2.995451,0.469807,03:59
1,2.460733,3.030260,0.463768,03:45
2,2.396108,3.016617,0.477053,03:33
3,2.337207,3.023128,0.468599,03:49


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.323862,3.094115,0.461353,03:42
1,2.424077,3.102745,0.452899,03:52
2,2.431175,3.094921,0.458937,03:44
3,2.300790,3.034491,0.467391,03:39


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.306098,3.078142,0.469807,04:05
1,2.328220,3.072494,0.472222,03:57
2,2.321225,3.139161,0.474638,03:41
3,2.256360,3.148902,0.468599,03:54


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.201885,3.094347,0.471014,03:40
1,2.259495,3.094810,0.467391,03:52
2,2.246244,3.109401,0.472222,03:48
3,2.195169,3.080960,0.468599,03:56


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.194278,3.183734,0.471014,03:36
1,2.265203,3.115763,0.471014,04:04
2,2.226325,3.157380,0.464976,03:46
3,2.120949,3.155853,0.468599,03:52


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.200794,3.179410,0.467391,03:54
1,2.286073,3.199004,0.464976,03:50
2,2.234743,3.234541,0.472222,04:04
3,2.135508,3.263991,0.462560,03:42


In [0]:
# save the best model

learn.save_encoder('ask-4-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [44]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[624, 1035, 1026, 1026, 16, 359, 298, 298, 27, 486, 31, 414, 414, 414, 414, 32, 37, 415, 864, 864, 37, 289, 289, 1035, 517, 92, 949, 49, 0, 517, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 53, 517, 53, 517, 864, 521, 61, 61, 62, 517, 76, 76, 76, 30, 289, 172, 517, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 517, 825, 80, 80, 80, 89, 1026, 808, 1026, 1026, 1026, 94, 94, 94, 517, 517, 517, 517, 517, 517, 135, 135, 135, 135, 289, 94, 94, 49, 265, 902, 265, 503, 185, 650, 864, 1035, 517, 864, 864, 289, 210, 182, 864, 289, 1035, 116, 517, 831, 265, 265, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 129, 131, 139, 140, 189, 788, 1035, 160, 178, 178, 164, 517, 291, 741, 741, 289, 864, 864, 864, 1035, 479, 178, 182, 741, 465, 864, 864, 289, 289, 352, 183, 517, 183, 183, 183, 183, 183, 183, 183, 183, 517, 240, 240, 111, 1035, 1035, 352, 30, 190, 190, 190, 190, 289, 289, 3,

In [45]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

206
0.23787528868360278
